In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import itertools
import os
import time
import csv
import math

from pathlib import Path
from tqdm.notebook import tqdm

import pydicom
from alignment import *

# this is from the contour.py file in the dicom-contour github
import contour

Below we have a dataframe with paths to a few places for each patient:
- Folder with CT info
- Folder with Dose info
- Folder with Plan info
- Folder with Contour info (masks)

In [2]:
data_path = Path('/data/users/ruddym/DOSE/plans_ck_old/')
patient_df = pd.read_csv(data_path/'patient_table.csv')

# let's look at a particular patient
idx = 54

Notice that when we peek into the CT folder, we have a bunch of ```dcm``` files.

In [3]:
CT_folder = patient_df.iloc[idx]['CT']

num_dcm_files = 0

for file in os.listdir(data_path/CT_folder):
    if file.split('.')[-1] == 'dcm':
        num_dcm_files += 1

print(f'There are {num_dcm_files} dicom files in this folder')

There are 419 dicom files in this folder


Let's peek at a particular ```dcm``` file using the pydicom package. This contains a lot of information about the patient and the CT scan, not just the CT scan pixels itself. There are standard codes for the information in a DICOM file (see [here](https://dicom.innolitics.com/ciods/cr-image) for reference).

In [4]:
file_no = 0
file = os.listdir(data_path/CT_folder)[file_no]

dicom_obj = pydicom.read_file(str(data_path/CT_folder/file))

In [5]:
# the metadata can be accessed via standard keys
print(dicom_obj.keys())

# we can extract the modality this way
dicom_obj["0008", "0060"]

dict_keys([(0008, 0005), (0008, 0008), (0008, 0016), (0008, 0018), (0008, 0020), (0008, 0021), (0008, 0022), (0008, 0023), (0008, 0030), (0008, 0031), (0008, 0032), (0008, 0033), (0008, 0050), (0008, 0060), (0008, 0070), (0008, 0080), (0008, 0081), (0008, 0090), (0008, 1010), (0008, 1030), (0008, 103e), (0008, 1048), (0008, 1090), (0008, 1110), (0008, 1140), (0008, 2112), (0009, 0010), (0010, 0010), (0010, 0020), (0010, 0021), (0010, 0030), (0010, 0032), (0010, 0040), (0010, 1010), (0010, 1040), (0010, 2154), (0018, 0015), (0018, 0050), (0018, 0060), (0018, 0090), (0018, 1000), (0018, 1020), (0018, 1030), (0018, 1100), (0018, 1110), (0018, 1111), (0018, 1120), (0018, 1130), (0018, 1140), (0018, 1150), (0018, 1151), (0018, 1152), (0018, 1160), (0018, 1170), (0018, 1190), (0018, 1200), (0018, 1201), (0018, 1210), (0018, 5100), (0019, 0010), (0019, 10b0), (0020, 000d), (0020, 000e), (0020, 0010), (0020, 0011), (0020, 0012), (0020, 0013), (0020, 0032), (0020, 0037), (0020, 0052), (0020, 10

(0008, 0060) Modality                            CS: 'CT'

In [6]:
# or do so directly
dicom_obj.Modality

'CT'

In [7]:
# this dicom file comes from UCSF
dicom_obj.InstitutionName

'UCSF'

Notably this file contains geometric information about the CT scan. Most of these values are in millimeters, referencing some origin in physical space. This origin should match the patient's associated dose, plan, and contour files as well. See [here](https://nipy.org/nibabel/dicom/dicom_orientation.html) and [here](https://dicom.nema.org/MEDICAL/DICOM/2014c/output/chtml/part03/sect_C.7.6.2.html#sect_C.7.6.2.1.1) for more information. This is necessary for reconstructing the CT scan into a 3D array and for aligning it with the dose data.

**NOTE:**
- The (x,y) coordinates of dicom files may refer to the *columns* and *rows* of the pixel array respectively. In other words the (*i*,*j*)-th voxel refers to pixel given by ```pixel_array[j,i]```.
- Here *z* is with respect to the patient, not physical orientation. Thus z ranges from the feet (lower values) to the head (higher values).

In [8]:
# x, y, z coordinates of the top left corner of the CT scan slice
dicom_obj.ImagePositionPatient

[-249.51171875, -470.51171875, 107.5]

In [9]:
# z coordinate of the slice
dicom_obj.SliceLocation

'107.5'

In [10]:
# physical distance between each pixel in mm
dicom_obj.PixelSpacing

[0.9765625, 0.9765625]

In [11]:
# physical "thickness" of the slice in mm
dicom_obj.SliceThickness

'1.0'

Let's play with some of the helper functions in the alignment file.

In [12]:
# gets all dicom files of a given modality
CT_slices = get_dicom_files(str(data_path/CT_folder), modality='CT')

# on top of this function we have one that sorts CT slices
slices = get_CT_ordered_slices(str(data_path/CT_folder))

# and on top of that we have one that returns the CT array
CT_array = get_CT_array(str(data_path/CT_folder))
type(CT_array)

numpy.ndarray

In [13]:
# note the 419 again
CT_array.shape

(512, 512, 419)

In [14]:
# get the physical spacing from a dicom file as floating point
spacing_from_dicom(slices[0])

[0.9765625, 0.9765625, 1.0]

In [15]:
# get the origin
origin_from_dicom(slices[0])

[-249.51171875, -470.51171875, 10.5]

In [16]:
# note the origin of the second slice
# should be reconstructable from Slice Thickness and first slice's origin
origin_from_dicom(slices[1])

[-249.51171875, -470.51171875, 11.5]

We can use the contour package from [here](https://github.com/KeremTurgutlu/dicom-contour/blob/master/dicom_contour/contour.py) to get the masks as 3D arrays. The various masks are of ROIs (Regions of Interest)

In [17]:
mask_folder = patient_df.iloc[idx]['RTst']
contour_data = get_dicom_files(str(data_path/mask_folder), modality='RTSTRUCT')

In [18]:
# these are the masks in the contour data
contour.get_roi_names(contour_data)

['Urethra',
 'Bladder',
 'PenileBulb',
 'Femur_R',
 'Femur_L',
 'Rectum',
 'Prostate']

In [19]:
# dealing with strings is tough, ROI naming is non-standard as well
# this function looks for an ROI named similarly to what you are looking for
suggest_roi_name(contour_data, 'Femur')

['Femur_R', 'Femur_L']

In [20]:
# get the index of the desired ROI in the ROI sequence
get_roi_index(contour_data, 'Femur')

/home/ruddym/dose_alignment/alignment.py:341: UserWarning: Contour Femur is not in contour dataset!
  warnings.warn(f"Contour {name} is not in contour dataset!")


In [21]:
# it must be exact!
get_roi_index(contour_data, 'Femur_R')

3

In [22]:
# convert the contour into a filled in binary mask with aligns with the CT scan
contour_fname =  get_dicom_files(str(data_path/mask_folder), modality='RTSTRUCT',
                                fname=True)
mask = contour.get_mask(str(data_path/CT_folder), str(data_path/mask_folder/contour_fname), 3)
type(mask)

numpy.ndarray

In [23]:
# note that the mask needs to be transposed to match CT array
mask.shape

(419, 512, 512)

In [24]:
np.transpose(mask, (1, 2, 0)).shape

(512, 512, 419)

Now let's take a look at the dose data. We also have geometric information about the dose, but note that we have one dose file with a 3D array rather than a collection of slices like for the CT scan.

In [25]:
dose_folder = patient_df.iloc[idx]['RTDOSE']
dose_data = get_dicom_files(str(data_path/dose_folder), modality='RTDOSE')

In [26]:
# this doesn't match the first CT slice exactly everywhere
# we'll have to align them later!
origin_from_dicom(dose_data)

[-214.35546875, -345.51171875, 10.5]

In [27]:
# it must be transposed to match
dose_array = np.transpose(dose_data.pixel_array, (1, 2, 0))

# it's a different shape too
dose_array.shape

(277, 437, 418)

The pixels of the dose array correspond to the intensity of radiation. We need certain metadata from the dose array to convert from these values to Grays.

In [28]:
# units in Grays
dose_data.DoseUnits

'GY'

In [29]:
# converts to Grays
dose_data.DoseGridScaling

'0.00072480354009'

In [30]:
print(f'The max pixel value of the dose array is {np.max(dose_array)}')

# dose array scaled to Grays
dose_array_grays = dose_array * dose_data.DoseGridScaling

print(f'The max value in Grays of the dose array is {np.max(dose_array_grays)}')

The max pixel value of the dose array is 65535
The max value in Grays of the dose array is 47.49999999979815


Often we want to normalize the dose array with respect to a certain value in Grays, the prescription dose. This way we can compare two patients receiving similar treatment but perhaps different intensity radiation.

In [31]:
Rx_dose_grays = 35
normalized_dose_array = get_dose_array(str(data_path/dose_folder), Rx_normalization=35)
print(f'The max value after normalization is {np.max(normalized_dose_array)}.')
print(f'The value of 1.0 in the array corresponds to {Rx_dose_grays} Grays.')

The max value after normalization is 1.35714285713709.
The value of 1.0 in the array corresponds to 35 Grays.


The most important function in the alignment file is the function that aligns a dose to the CT array. It may take a non-trivial amount of time to do this aligning. The following aligns the dose array *to* the CT array, meaning that we create a new dose array rather than a new CT array. To align in the other direction simply replace the dose info with the CT info and vice versa.

**NOTE:**
- This currently only works with patients that have image orientation: ```[1,0,0,0,1,0]```

In [32]:
CT_shape, CT_origin, CT_spacing = get_CT_voxel_data(str(data_path/CT_folder))
dose_shape, dose_origin, dose_spacing = get_dose_voxel_data(str(data_path/dose_folder))

start = time.time()
aligned_dose_array = align_3d_arrays(dose_origin, dose_spacing, dose_array,
                                        CT_origin, CT_spacing, CT_shape)
end = time.time()

print(f'This took {end - start} seconds')

This took 95.75315380096436 seconds


In [33]:
# now the shapes match!
aligned_dose_array.shape, CT_shape

((512, 512, 419), (512, 512, 419))

Finally there are plan files which contain information about the plan made by the clinician before radiation treatment.

In [34]:
plan_folder = patient_df.iloc[idx]['RTPLAN']

plan = get_dicom_files(str(data_path/plan_folder), modality='RTPLAN')

This function looks for the prescription dose of the target ROI in the plan file. It is very heuristic and may not work well for another patient set.

In [35]:
Rx_dose_from_plan(plan, target_names=['target'])

38.0